In [2]:
import os
import sys
import pandas
import time
import csv
import json
import openpyxl
from IPython.display import display, HTML

#because shitty duplicate keys, now there is a global duplicate count, and every object whose key is duplicate, like "ruler_term" has now it's own duplicate number
global global_dupl_count
global_dupl_count=0
def dict_raise_on_duplicates(ordered_pairs):
    """Reject duplicate keys."""
    global global_dupl_count
    d = {}
    for k, v in ordered_pairs:
        if k in d:
            global_dupl_count=global_dupl_count+1
            d[k+str(global_dupl_count)] = v
        else:
            d[k] = v
    return d

#filepath = "../Imperator/save games/"
filepath = "D:/python/Imperator/01_json/mp_rome_grand_camp/"
filename = "mp_rome_grand_camp14-02-2024_01h50"
savegame = filepath+filename+".json"

with open(savegame, encoding="utf-8") as source:
    source=json.load(source,object_pairs_hook=dict_raise_on_duplicates)
#with open("imperator/mappings/countries_names.csv") as countries_names:
#    countries_names = pandas.read_csv(countries_names, sep=";", header=None)
#    countries_names.columns = ["country_name_key", "country_name"]
        

In [3]:
from imperator import country_analyses
from imperator import state_analyses
from imperator import province_analyses

from imperator import pop_analyses
from imperator import char_analyses
from imperator import rulerterm_analyses

from imperator import war_analyses
from imperator import battle_analyses
from imperator import family_analyses




countries = country_analyses.extract(source)
countries["date"]=source["date"]
states = state_analyses.extract(source)
states["date"]=source["date"]
provinces = province_analyses.extract(source)[0]
provinces["date"]=source["date"]

pops = pop_analyses.extract(source).join(province_analyses.extract(source)[1])

chars = char_analyses.extract(source)
rulerterms = rulerterm_analyses.extract(source)
families = family_analyses.extract(source)
wars = war_analyses.extract(source)
battles = battle_analyses.extract(source)

#countries["country_name_key"]=countries["historical"]
#countries["total_income"]=round(countries["economy_lastmonth_tax_income"]+countries["economy_lastmonth_tax_income"]+countries["economy_lastmonth_tax_income"])
#countries = countries.sort_values("total_holdings",ascending=False).query('total_income > 10 & total_holdings > 0')

# with pandas.ExcelWriter("D:/python/"+filename+".xlsx") as writer:

    # rulerterms.to_excel(writer, sheet_name='rulers')  
    # countries.to_excel(writer, sheet_name='countries')  
    # provinces.to_excel(writer, sheet_name='provinces')  
    # pops.to_excel(writer, sheet_name='pops')  
    #chars.to_excel(writer, sheet_name='chars')  
    # wars.to_excel(writer, sheet_name='wars')
    # battles.to_excel(writer, sheet_name='battles')

#freeze_header(countries[countries.total_population!=0].sort_values(by=["total_population"],ascending=False))
# freeze_header(provinces.sort_values(by=["pop_count"],ascending=False))


In [4]:
from IPython.display import Markdown as md
# md("<span style= 'background:"+color1+";color:"+color2+";font-weight: bold;'> color1 </span>")

# output=countries[countries["total_population"]>100].sort_values(by=["total_population"],ascending=False)
# output = states.to_html()
# md (output)

In [5]:
from imperator import naming
# print(source["legion_manager"].keys())
legions=source["legion_manager"]["legions"]
history=source["legion_manager"]["history"]
    
result=[]
events=[]

for legion_id in legions.keys():
    if legions[legion_id]!="none":
        legion=legions[legion_id]
        legion_commanders=""
        legion_units=""
        battles_lost = 0
        battles_won = 0
        if "history" in legion:
            legion_history=legion["history"]
            errors=0
            facts = history[str(legion_history)]
            for fact in facts.keys():
                if "battles_lost" in fact: battles_lost = facts["battles_lost"]
                if "battles_won" in fact: battles_won = facts["battles_won"]
                if "battles_won" in fact: battles_won = facts["battles_won"]
                if "events" in fact:
                    for i in range(len(facts["events"])):
                        event=facts["events"][i]
                        location=""
                        character=""
                        key=""
                        date=""
                        if "location" in event: location = event["location"]
                        if "character" in event: character = event["character"]
                        if "key" in event: key = event["key"]
                        if "date" in event: date = event["date"]
                        try: location = naming.name(location, "geography")
                        except: location = "PROV"+str(location)
                        try: events.append([legion_id,location,chars.at[str(character),"first_name"]+" "+chars.at[str(character),"family_name"]+" ("+str(character)+")",key,date])
                        except: events.append([legion_id,location,character,key,date])
                    # print(facts["events"][0])
        if "commanders" in legion["commanders"]: legion_commanders=legion["commanders"]["commanders"]
        if "origin" in legion: legion_origin=legion["origin"]
        if "region" in legion: legion_region=legion["region"]
        if "units" in legion: legion_units=legion["units"]

        result.append([legion_id,legion_history,legion_commanders,legion_origin,legion_region,legion_units,battles_lost,battles_won,errors])
        # else: print(family)

# print(legions["1"])
# print(history["1"]["events"])
output=pandas.DataFrame(result)
events.columns = ["legion_id","legion_history","legion_commanders","legion_origin","legion_region","legion_units","battles_lost","battles_won","errors"]
output=output.to_html()
events=pandas.DataFrame(events)
events.columns = ["legion_id","location","character","key","date"]

# output = events.to_html()

md (output)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
      <th>2</th>
      <th>3</th>
      <th>4</th>
      <th>5</th>
      <th>6</th>
      <th>7</th>
      <th>8</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>100663296</td>
      <td>152</td>
      <td></td>
      <td>891</td>
      <td>arabia_felix_region</td>
      <td></td>
      <td>3</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>1</td>
      <td>1</td>
      <td>[180148]</td>
      <td>26</td>
      <td>macedonia_region</td>
      <td>[301990315]</td>
      <td>18</td>
      <td>147</td>
      <td>0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>16777218</td>
      <td>16777218</td>
      <td></td>
      <td>968</td>
      <td>gedrosia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>3</th>
      <td>150994947</td>
      <td>270</td>
      <td></td>
      <td>1024</td>
      <td>gedrosia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>4</th>
      <td>67108868</td>
      <td>253</td>
      <td></td>
      <td>1015</td>
      <td>media_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>5</th>
      <td>67108869</td>
      <td>478</td>
      <td></td>
      <td>1211</td>
      <td>lower_egypt_region</td>
      <td></td>
      <td>6</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>6</th>
      <td>50331654</td>
      <td>193</td>
      <td></td>
      <td>941</td>
      <td>parthia_region</td>
      <td></td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>7</th>
      <td>67108871</td>
      <td>227</td>
      <td></td>
      <td>979</td>
      <td>parthia_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>8</th>
      <td>218103816</td>
      <td>619</td>
      <td>[182251, 182250, 182686, 182978]</td>
      <td>1350</td>
      <td>gedrosia_region</td>
      <td>[1342177529]</td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>9</th>
      <td>50331657</td>
      <td>240</td>
      <td></td>
      <td>998</td>
      <td>moesia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>10</th>
      <td>33554442</td>
      <td>219</td>
      <td></td>
      <td>971</td>
      <td>moesia_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>11</th>
      <td>100663307</td>
      <td>95</td>
      <td></td>
      <td>829</td>
      <td>asia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>12</th>
      <td>33554444</td>
      <td>33554435</td>
      <td>[178779, 178940, 179801, 181561]</td>
      <td>460</td>
      <td>dravida_region</td>
      <td>[1107296470, 1040187630]</td>
      <td>33</td>
      <td>79</td>
      <td>0</td>
    </tr>
    <tr>
      <th>13</th>
      <td>33554445</td>
      <td>16777297</td>
      <td></td>
      <td>819</td>
      <td>nubia_region</td>
      <td></td>
      <td>6</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>14</th>
      <td>150994958</td>
      <td>490</td>
      <td></td>
      <td>1227</td>
      <td>gandhara_region</td>
      <td></td>
      <td>10</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>15</th>
      <td>33554447</td>
      <td>610</td>
      <td>[178292, 179345, 180478, 178977]</td>
      <td>451</td>
      <td>aparanta_region</td>
      <td>[452985162]</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>16</th>
      <td>67108880</td>
      <td>402</td>
      <td></td>
      <td>1156</td>
      <td>gedrosia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>17</th>
      <td>67108881</td>
      <td>327</td>
      <td></td>
      <td>1078</td>
      <td>parthia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>18</th>
      <td>50331666</td>
      <td>16777375</td>
      <td></td>
      <td>907</td>
      <td>dravida_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>19</th>
      <td>134217747</td>
      <td>627</td>
      <td>[183160, 183159]</td>
      <td>1358</td>
      <td>gandhara_region</td>
      <td>[1694498979]</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>20</th>
      <td>33554452</td>
      <td>16777260</td>
      <td></td>
      <td>1200</td>
      <td>syria_region</td>
      <td></td>
      <td>13</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>21</th>
      <td>167772181</td>
      <td>455</td>
      <td>[179502, 179110, 179711]</td>
      <td>112</td>
      <td>africa_region</td>
      <td>[1157628056, 838860959]</td>
      <td>0</td>
      <td>19</td>
      <td>0</td>
    </tr>
    <tr>
      <th>22</th>
      <td>201326614</td>
      <td>499</td>
      <td></td>
      <td>1235</td>
      <td>caledonia_region</td>
      <td></td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>23</th>
      <td>167772183</td>
      <td>16777721</td>
      <td></td>
      <td>655</td>
      <td>caledonia_region</td>
      <td></td>
      <td>4</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>24</th>
      <td>134217752</td>
      <td>16777691</td>
      <td>[179375, 179247, 179458, 182440]</td>
      <td>460</td>
      <td>karnata_region</td>
      <td>[2516582497]</td>
      <td>3</td>
      <td>6</td>
      <td>0</td>
    </tr>
    <tr>
      <th>25</th>
      <td>83886105</td>
      <td>50331868</td>
      <td>[179145, 178898, 179276, 180542]</td>
      <td>519</td>
      <td>scythia_region</td>
      <td>[2533359696, 1241514094, 3204448352]</td>
      <td>28</td>
      <td>162</td>
      <td>0</td>
    </tr>
    <tr>
      <th>26</th>
      <td>33554458</td>
      <td>226</td>
      <td></td>
      <td>975</td>
      <td>fezzan_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>27</th>
      <td>134217755</td>
      <td>628</td>
      <td>[179059]</td>
      <td>1359</td>
      <td>nubia_region</td>
      <td>[1761607841]</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>28</th>
      <td>16777244</td>
      <td>16777258</td>
      <td></td>
      <td>768</td>
      <td>punt_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>29</th>
      <td>16777245</td>
      <td>180</td>
      <td></td>
      <td>924</td>
      <td>greece_region</td>
      <td></td>
      <td>4</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>30</th>
      <td>16777246</td>
      <td>16777261</td>
      <td></td>
      <td>769</td>
      <td>himalayan_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>31</th>
      <td>150994975</td>
      <td>580</td>
      <td></td>
      <td>1315</td>
      <td>albania_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>32</th>
      <td>134217760</td>
      <td>450</td>
      <td>[177905, 178566, 181890, 182992]</td>
      <td>720</td>
      <td>karnata_region</td>
      <td>[1811939426]</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>33</th>
      <td>16777249</td>
      <td>16777264</td>
      <td></td>
      <td>765</td>
      <td>avanti_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>34</th>
      <td>167772194</td>
      <td>430</td>
      <td></td>
      <td>1173</td>
      <td>aparanta_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>35</th>
      <td>35</td>
      <td>51</td>
      <td></td>
      <td>763</td>
      <td>cappadocia_pontica_region</td>
      <td></td>
      <td>2</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>36</th>
      <td>33554468</td>
      <td>16777391</td>
      <td></td>
      <td>919</td>
      <td>moesia_region</td>
      <td></td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>37</th>
      <td>285212709</td>
      <td>415</td>
      <td>[179374, 179705, 181357, 181947]</td>
      <td>94</td>
      <td>persis_region</td>
      <td>[1442840832]</td>
      <td>51</td>
      <td>138</td>
      <td>0</td>
    </tr>
    <tr>
      <th>38</th>
      <td>184549414</td>
      <td>604</td>
      <td>[176430, 178685, 182620, 182995]</td>
      <td>731</td>
      <td>pracya_region</td>
      <td>[2483028162]</td>
      <td>4</td>
      <td>7</td>
      <td>0</td>
    </tr>
    <tr>
      <th>39</th>
      <td>100663335</td>
      <td>16777385</td>
      <td></td>
      <td>929</td>
      <td>arabia_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>40</th>
      <td>167772200</td>
      <td>372</td>
      <td></td>
      <td>1136</td>
      <td>ariana_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>41</th>
      <td>16777258</td>
      <td>100</td>
      <td></td>
      <td>834</td>
      <td>dravida_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>42</th>
      <td>134217771</td>
      <td>343</td>
      <td>[177265, 181269, 180974]</td>
      <td>112</td>
      <td>magna_graecia_region</td>
      <td>[1140850924, 704643364]</td>
      <td>1</td>
      <td>74</td>
      <td>0</td>
    </tr>
    <tr>
      <th>43</th>
      <td>150994988</td>
      <td>131</td>
      <td></td>
      <td>868</td>
      <td>madhyadesa_region</td>
      <td></td>
      <td>4</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>44</th>
      <td>117440557</td>
      <td>453</td>
      <td></td>
      <td>1190</td>
      <td>persis_region</td>
      <td></td>
      <td>4</td>
      <td>3</td>
      <td>0</td>
    </tr>
    <tr>
      <th>45</th>
      <td>67108910</td>
      <td>225</td>
      <td></td>
      <td>976</td>
      <td>gedrosia_region</td>
      <td></td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>46</th>
      <td>67108911</td>
      <td>33554545</td>
      <td></td>
      <td>993</td>
      <td>mesopotamia_region</td>
      <td></td>
      <td>0</td>
      <td>4</td>
      <td>0</td>
    </tr>
    <tr>
      <th>47</th>
      <td>48</td>
      <td>80</td>
      <td></td>
      <td>816</td>
      <td>bactriana_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>48</th>
      <td>16777265</td>
      <td>89</td>
      <td></td>
      <td>825</td>
      <td>punt_region</td>
      <td></td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>49</th>
      <td>50</td>
      <td>82</td>
      <td></td>
      <td>820</td>
      <td>palestine_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>50</th>
      <td>50331699</td>
      <td>242</td>
      <td></td>
      <td>1004</td>
      <td>fezzan_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>51</th>
      <td>184549428</td>
      <td>33554731</td>
      <td></td>
      <td>1096</td>
      <td>media_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>52</th>
      <td>16777269</td>
      <td>174</td>
      <td></td>
      <td>917</td>
      <td>vindhyaprstha_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>53</th>
      <td>50331702</td>
      <td>16777416</td>
      <td></td>
      <td>956</td>
      <td>vindhyaprstha_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>54</th>
      <td>50331703</td>
      <td>135</td>
      <td></td>
      <td>870</td>
      <td>asia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>55</th>
      <td>83886136</td>
      <td>203</td>
      <td></td>
      <td>957</td>
      <td>arabia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>56</th>
      <td>100663353</td>
      <td>214</td>
      <td></td>
      <td>966</td>
      <td>bithynia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>57</th>
      <td>33554490</td>
      <td>157</td>
      <td></td>
      <td>902</td>
      <td>madhyadesa_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>58</th>
      <td>100663355</td>
      <td>218</td>
      <td></td>
      <td>969</td>
      <td>armenia_region</td>
      <td></td>
      <td>3</td>
      <td>2</td>
      <td>0</td>
    </tr>
    <tr>
      <th>59</th>
      <td>67108924</td>
      <td>141</td>
      <td></td>
      <td>873</td>
      <td>lower_egypt_region</td>
      <td></td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>60</th>
      <td>83886141</td>
      <td>509</td>
      <td></td>
      <td>1245</td>
      <td>himalayan_region</td>
      <td></td>
      <td>7</td>
      <td>15</td>
      <td>0</td>
    </tr>
    <tr>
      <th>61</th>
      <td>83886142</td>
      <td>16777404</td>
      <td></td>
      <td>933</td>
      <td>arabia_felix_region</td>
      <td></td>
      <td>2</td>
      <td>2</td>
      <td>0</td>
    </tr>
    <tr>
      <th>62</th>
      <td>50331711</td>
      <td>281</td>
      <td></td>
      <td>1036</td>
      <td>himalayan_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>63</th>
      <td>67108928</td>
      <td>358</td>
      <td></td>
      <td>1104</td>
      <td>sogdiana_region</td>
      <td></td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>64</th>
      <td>65</td>
      <td>137</td>
      <td></td>
      <td>862</td>
      <td>bithynia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>65</th>
      <td>66</td>
      <td>138</td>
      <td>[181674, 179130, 182103, 180682]</td>
      <td>112</td>
      <td>central_italy_region</td>
      <td>[738197759, 469762557, 1275068640]</td>
      <td>1</td>
      <td>284</td>
      <td>0</td>
    </tr>
    <tr>
      <th>66</th>
      <td>67</td>
      <td>139</td>
      <td></td>
      <td>874</td>
      <td>mesopotamia_region</td>
      <td></td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>67</th>
      <td>150995012</td>
      <td>16777531</td>
      <td></td>
      <td>1142</td>
      <td>sogdiana_region</td>
      <td></td>
      <td>8</td>
      <td>3</td>
      <td>0</td>
    </tr>
    <tr>
      <th>68</th>
      <td>69</td>
      <td>142</td>
      <td></td>
      <td>869</td>
      <td>karnata_region</td>
      <td></td>
      <td>6</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>69</th>
      <td>83886150</td>
      <td>432</td>
      <td></td>
      <td>1171</td>
      <td>himalayan_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>70</th>
      <td>67108935</td>
      <td>16777359</td>
      <td></td>
      <td>898</td>
      <td>maru_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>71</th>
      <td>67108936</td>
      <td>230</td>
      <td></td>
      <td>982</td>
      <td>bithynia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>72</th>
      <td>16777289</td>
      <td>332</td>
      <td></td>
      <td>1084</td>
      <td>media_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>73</th>
      <td>50331722</td>
      <td>111</td>
      <td>[182767, 182768, 182878, 182879]</td>
      <td>847</td>
      <td>ariana_region</td>
      <td>[973078823]</td>
      <td>6</td>
      <td>13</td>
      <td>0</td>
    </tr>
    <tr>
      <th>74</th>
      <td>75</td>
      <td>154</td>
      <td></td>
      <td>878</td>
      <td>albania_region</td>
      <td></td>
      <td>3</td>
      <td>6</td>
      <td>0</td>
    </tr>
    <tr>
      <th>75</th>
      <td>16777292</td>
      <td>16777289</td>
      <td></td>
      <td>883</td>
      <td>arabia_region</td>
      <td></td>
      <td>3</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>76</th>
      <td>77</td>
      <td>158</td>
      <td></td>
      <td>903</td>
      <td>fezzan_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>77</th>
      <td>67108942</td>
      <td>527</td>
      <td></td>
      <td>1265</td>
      <td>ariana_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>78</th>
      <td>79</td>
      <td>164</td>
      <td></td>
      <td>892</td>
      <td>lower_nubia_region</td>
      <td></td>
      <td>6</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>79</th>
      <td>67108944</td>
      <td>393</td>
      <td></td>
      <td>497</td>
      <td>gandhara_region</td>
      <td></td>
      <td>3</td>
      <td>4</td>
      <td>0</td>
    </tr>
    <tr>
      <th>80</th>
      <td>81</td>
      <td>166</td>
      <td></td>
      <td>908</td>
      <td>bithynia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>81</th>
      <td>16777298</td>
      <td>231</td>
      <td></td>
      <td>984</td>
      <td>arabia_region</td>
      <td></td>
      <td>1</td>
      <td>11</td>
      <td>0</td>
    </tr>
    <tr>
      <th>82</th>
      <td>16777299</td>
      <td>16777384</td>
      <td></td>
      <td>901</td>
      <td>avanti_region</td>
      <td></td>
      <td>10</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>83</th>
      <td>67108948</td>
      <td>16777403</td>
      <td></td>
      <td>1050</td>
      <td>fezzan_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>84</th>
      <td>85</td>
      <td>177</td>
      <td></td>
      <td>921</td>
      <td>gedrosia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>85</th>
      <td>86</td>
      <td>184</td>
      <td></td>
      <td>927</td>
      <td>cappadocia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>86</th>
      <td>33554519</td>
      <td>228</td>
      <td>[179601, 181863, 182396, 180095]</td>
      <td>595</td>
      <td>asia_region</td>
      <td>[469762102]</td>
      <td>3</td>
      <td>2</td>
      <td>0</td>
    </tr>
    <tr>
      <th>87</th>
      <td>16777304</td>
      <td>205</td>
      <td></td>
      <td>958</td>
      <td>baetica_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>88</th>
      <td>89</td>
      <td>202</td>
      <td></td>
      <td>943</td>
      <td>karnata_region</td>
      <td></td>
      <td>6</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>89</th>
      <td>90</td>
      <td>208</td>
      <td></td>
      <td>960</td>
      <td>contestania_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>90</th>
      <td>134217819</td>
      <td>392</td>
      <td></td>
      <td>1149</td>
      <td>gedrosia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>91</th>
      <td>150995036</td>
      <td>298</td>
      <td></td>
      <td>1052</td>
      <td>dacia_region</td>
      <td></td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>92</th>
      <td>93</td>
      <td>211</td>
      <td></td>
      <td>953</td>
      <td>dravida_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>93</th>
      <td>50331742</td>
      <td>562</td>
      <td></td>
      <td>1299</td>
      <td>gandhara_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>94</th>
      <td>100663391</td>
      <td>376</td>
      <td></td>
      <td>1139</td>
      <td>lower_nubia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>95</th>
      <td>33554528</td>
      <td>16777461</td>
      <td></td>
      <td>1097</td>
      <td>gedrosia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>96</th>
      <td>97</td>
      <td>237</td>
      <td></td>
      <td>995</td>
      <td>cappadocia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>97</th>
      <td>167772258</td>
      <td>413</td>
      <td></td>
      <td>1163</td>
      <td>gedrosia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>98</th>
      <td>99</td>
      <td>239</td>
      <td></td>
      <td>996</td>
      <td>himalayan_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>99</th>
      <td>100</td>
      <td>241</td>
      <td></td>
      <td>992</td>
      <td>greece_region</td>
      <td></td>
      <td>3</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>100</th>
      <td>101</td>
      <td>244</td>
      <td></td>
      <td>1006</td>
      <td>punt_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>101</th>
      <td>67108966</td>
      <td>16777719</td>
      <td></td>
      <td>1304</td>
      <td>moesia_s_region</td>
      <td></td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>102</th>
      <td>16777319</td>
      <td>348</td>
      <td></td>
      <td>1091</td>
      <td>himalayan_region</td>
      <td></td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>103</th>
      <td>117440616</td>
      <td>353</td>
      <td></td>
      <td>1100</td>
      <td>dacia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>104</th>
      <td>67108969</td>
      <td>598</td>
      <td></td>
      <td>1331</td>
      <td>lower_egypt_region</td>
      <td></td>
      <td>21</td>
      <td>3</td>
      <td>0</td>
    </tr>
    <tr>
      <th>105</th>
      <td>83886186</td>
      <td>16777610</td>
      <td></td>
      <td>1184</td>
      <td>gedrosia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>106</th>
      <td>33554539</td>
      <td>273</td>
      <td></td>
      <td>1029</td>
      <td>palestine_region</td>
      <td></td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>107</th>
      <td>134217836</td>
      <td>339</td>
      <td></td>
      <td>1085</td>
      <td>greece_region</td>
      <td></td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>108</th>
      <td>16777325</td>
      <td>16777491</td>
      <td></td>
      <td>1160</td>
      <td>assyria_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>109</th>
      <td>33554542</td>
      <td>296</td>
      <td></td>
      <td>1048</td>
      <td>atlas_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>110</th>
      <td>67108975</td>
      <td>294</td>
      <td></td>
      <td>1037</td>
      <td>armenia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>111</th>
      <td>33554544</td>
      <td>391</td>
      <td></td>
      <td>1107</td>
      <td>media_region</td>
      <td></td>
      <td>2</td>
      <td>9</td>
      <td>0</td>
    </tr>
    <tr>
      <th>112</th>
      <td>67108977</td>
      <td>511</td>
      <td>[179195, 181381, 177568, 181080]</td>
      <td>1247</td>
      <td>tibet_region</td>
      <td>[1912602686]</td>
      <td>6</td>
      <td>5</td>
      <td>0</td>
    </tr>
    <tr>
      <th>113</th>
      <td>114</td>
      <td>301</td>
      <td></td>
      <td>1053</td>
      <td>taurica_region</td>
      <td></td>
      <td>1</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>114</th>
      <td>16777331</td>
      <td>374</td>
      <td></td>
      <td>1137</td>
      <td>nubia_region</td>
      <td></td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>115</th>
      <td>33554548</td>
      <td>330</td>
      <td>[179485]</td>
      <td>26</td>
      <td>thrace_region</td>
      <td>[452984969]</td>
      <td>9</td>
      <td>62</td>
      <td>0</td>
    </tr>
    <tr>
      <th>116</th>
      <td>16777333</td>
      <td>512</td>
      <td>[178100, 182379, 182525, 181962]</td>
      <td>1249</td>
      <td>madhyadesa_region</td>
      <td>[1677721677]</td>
      <td>7</td>
      <td>26</td>
      <td>0</td>
    </tr>
    <tr>
      <th>117</th>
      <td>118</td>
      <td>307</td>
      <td></td>
      <td>1058</td>
      <td>gedrosia_region</td>
      <td></td>
      <td>3</td>
      <td>3</td>
      <td>0</td>
    </tr>
    <tr>
      <th>118</th>
      <td>33554551</td>
      <td>406</td>
      <td></td>
      <td>1159</td>
      <td>pannonia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>119</th>
      <td>120</td>
      <td>309</td>
      <td></td>
      <td>1060</td>
      <td>armenia_region</td>
      <td></td>
      <td>0</td>
      <td>5</td>
      <td>0</td>
    </tr>
    <tr>
      <th>120</th>
      <td>33554553</td>
      <td>16777528</td>
      <td></td>
      <td>1067</td>
      <td>gallaecia_region</td>
      <td></td>
      <td>4</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>121</th>
      <td>16777338</td>
      <td>320</td>
      <td></td>
      <td>1069</td>
      <td>palestine_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>122</th>
      <td>50331771</td>
      <td>16777369</td>
      <td></td>
      <td>1076</td>
      <td>britain_region</td>
      <td></td>
      <td>5</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>123</th>
      <td>124</td>
      <td>317</td>
      <td></td>
      <td>1056</td>
      <td>lower_nubia_region</td>
      <td></td>
      <td>4</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>124</th>
      <td>83886205</td>
      <td>526</td>
      <td>[175591, 178887, 175803]</td>
      <td>1264</td>
      <td>gandhara_region</td>
      <td>[1358954579]</td>
      <td>5</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>125</th>
      <td>16777342</td>
      <td>33554763</td>
      <td></td>
      <td>1083</td>
      <td>lusitania_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>126</th>
      <td>33554559</td>
      <td>599</td>
      <td></td>
      <td>1332</td>
      <td>bactriana_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>127</th>
      <td>117440640</td>
      <td>561</td>
      <td>[175204, 180137, 179572, 182856]</td>
      <td>731</td>
      <td>burma_region</td>
      <td>[1375731915]</td>
      <td>3</td>
      <td>42</td>
      <td>0</td>
    </tr>
    <tr>
      <th>128</th>
      <td>33554561</td>
      <td>16777500</td>
      <td></td>
      <td>1077</td>
      <td>punt_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>129</th>
      <td>67108994</td>
      <td>585</td>
      <td></td>
      <td>1319</td>
      <td>macedonia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>130</th>
      <td>83886211</td>
      <td>471</td>
      <td></td>
      <td>1208</td>
      <td>gedrosia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>131</th>
      <td>100663428</td>
      <td>468</td>
      <td></td>
      <td>1204</td>
      <td>persis_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>132</th>
      <td>16777349</td>
      <td>16777561</td>
      <td></td>
      <td>1093</td>
      <td>baetica_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>133</th>
      <td>33554566</td>
      <td>419</td>
      <td></td>
      <td>1168</td>
      <td>media_region</td>
      <td></td>
      <td>3</td>
      <td>2</td>
      <td>0</td>
    </tr>
    <tr>
      <th>134</th>
      <td>135</td>
      <td>347</td>
      <td></td>
      <td>1092</td>
      <td>taurica_region</td>
      <td></td>
      <td>3</td>
      <td>2</td>
      <td>0</td>
    </tr>
    <tr>
      <th>135</th>
      <td>100663434</td>
      <td>480</td>
      <td>[177115, 180976, 180869, 179236]</td>
      <td>1216</td>
      <td>ariana_region</td>
      <td></td>
      <td>7</td>
      <td>19</td>
      <td>0</td>
    </tr>
    <tr>
      <th>136</th>
      <td>33554571</td>
      <td>369</td>
      <td></td>
      <td>1110</td>
      <td>gedrosia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>137</th>
      <td>16777356</td>
      <td>16777580</td>
      <td></td>
      <td>1134</td>
      <td>dacia_region</td>
      <td></td>
      <td>3</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>138</th>
      <td>16777357</td>
      <td>16777581</td>
      <td></td>
      <td>1148</td>
      <td>taurica_region</td>
      <td></td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>139</th>
      <td>142</td>
      <td>367</td>
      <td></td>
      <td>1109</td>
      <td>ariana_region</td>
      <td></td>
      <td>2</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>140</th>
      <td>16777359</td>
      <td>370</td>
      <td></td>
      <td>1106</td>
      <td>pracya_region</td>
      <td></td>
      <td>2</td>
      <td>3</td>
      <td>0</td>
    </tr>
    <tr>
      <th>141</th>
      <td>33554576</td>
      <td>488</td>
      <td></td>
      <td>1226</td>
      <td>lower_nubia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>142</th>
      <td>167772305</td>
      <td>501</td>
      <td></td>
      <td>1237</td>
      <td>sogdiana_region</td>
      <td></td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>143</th>
      <td>50331794</td>
      <td>421</td>
      <td>[176534, 178879, 181476, 178495]</td>
      <td>112</td>
      <td>cisalpine_gaul_region</td>
      <td>[1258291338, 1593835635]</td>
      <td>0</td>
      <td>52</td>
      <td>0</td>
    </tr>
    <tr>
      <th>144</th>
      <td>16777364</td>
      <td>395</td>
      <td>[179680, 178500, 180457, 182454]</td>
      <td>708</td>
      <td>fezzan_region</td>
      <td></td>
      <td>4</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>145</th>
      <td>83886229</td>
      <td>329</td>
      <td>[174934]</td>
      <td>26</td>
      <td>greece_region</td>
      <td>[3137339438]</td>
      <td>6</td>
      <td>34</td>
      <td>0</td>
    </tr>
    <tr>
      <th>146</th>
      <td>83886231</td>
      <td>16777676</td>
      <td></td>
      <td>290</td>
      <td>nubia_region</td>
      <td></td>
      <td>11</td>
      <td>103</td>
      <td>0</td>
    </tr>
    <tr>
      <th>147</th>
      <td>83886233</td>
      <td>16777616</td>
      <td></td>
      <td>1164</td>
      <td>parthia_region</td>
      <td></td>
      <td>4</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>148</th>
      <td>154</td>
      <td>390</td>
      <td></td>
      <td>1144</td>
      <td>mauretainia_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>149</th>
      <td>67109019</td>
      <td>16777720</td>
      <td></td>
      <td>1240</td>
      <td>lower_egypt_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>150</th>
      <td>16777372</td>
      <td>429</td>
      <td></td>
      <td>1172</td>
      <td>gedrosia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>151</th>
      <td>50331805</td>
      <td>469</td>
      <td>[179204, 177718, 180304, 179055]</td>
      <td>283</td>
      <td>punt_region</td>
      <td>[50332124, 1996488900, 1040187549]</td>
      <td>58</td>
      <td>226</td>
      <td>0</td>
    </tr>
    <tr>
      <th>152</th>
      <td>33554590</td>
      <td>613</td>
      <td></td>
      <td>1346</td>
      <td>gandhara_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>153</th>
      <td>16777375</td>
      <td>491</td>
      <td></td>
      <td>1197</td>
      <td>himalayan_region</td>
      <td></td>
      <td>0</td>
      <td>6</td>
      <td>0</td>
    </tr>
    <tr>
      <th>154</th>
      <td>16777376</td>
      <td>16777709</td>
      <td></td>
      <td>1231</td>
      <td>arabia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>155</th>
      <td>33554594</td>
      <td>621</td>
      <td>[180575, 180574, 181281, 178944]</td>
      <td>1238</td>
      <td>himalayan_region</td>
      <td>[1962934332]</td>
      <td>14</td>
      <td>6</td>
      <td>0</td>
    </tr>
    <tr>
      <th>156</th>
      <td>164</td>
      <td>513</td>
      <td>[182277, 179116, 182380, 182278]</td>
      <td>1250</td>
      <td>maru_region</td>
      <td></td>
      <td>2</td>
      <td>6</td>
      <td>0</td>
    </tr>
    <tr>
      <th>157</th>
      <td>165</td>
      <td>514</td>
      <td>[177498, 179512, 179955, 180983]</td>
      <td>1251</td>
      <td>parthia_region</td>
      <td>[1845493943]</td>
      <td>2</td>
      <td>6</td>
      <td>0</td>
    </tr>
    <tr>
      <th>158</th>
      <td>16777382</td>
      <td>612</td>
      <td></td>
      <td>1345</td>
      <td>sogdiana_region</td>
      <td></td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>159</th>
      <td>167</td>
      <td>515</td>
      <td>[175385, 181382, 181383, 181772]</td>
      <td>1252</td>
      <td>gedrosia_region</td>
      <td>[1879048328]</td>
      <td>13</td>
      <td>7</td>
      <td>0</td>
    </tr>
    <tr>
      <th>160</th>
      <td>33554600</td>
      <td>408</td>
      <td>[181899]</td>
      <td>26</td>
      <td>bithynia_region</td>
      <td>[2080374837]</td>
      <td>2</td>
      <td>27</td>
      <td>0</td>
    </tr>
    <tr>
      <th>161</th>
      <td>169</td>
      <td>517</td>
      <td>[179280, 181774, 182381, 181967]</td>
      <td>1254</td>
      <td>vindhyaprstha_region</td>
      <td>[1526726711]</td>
      <td>8</td>
      <td>15</td>
      <td>0</td>
    </tr>
    <tr>
      <th>162</th>
      <td>50331818</td>
      <td>623</td>
      <td>[182804]</td>
      <td>1354</td>
      <td>gedrosia_region</td>
      <td>[2902458418]</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>163</th>
      <td>33554603</td>
      <td>622</td>
      <td></td>
      <td>1353</td>
      <td>upper_egypt_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>164</th>
      <td>172</td>
      <td>520</td>
      <td></td>
      <td>1248</td>
      <td>aparanta_region</td>
      <td></td>
      <td>2</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>165</th>
      <td>16777389</td>
      <td>495</td>
      <td>[179486]</td>
      <td>26</td>
      <td>moesia_region</td>
      <td>[1392509160]</td>
      <td>1</td>
      <td>10</td>
      <td>0</td>
    </tr>
    <tr>
      <th>166</th>
      <td>33554606</td>
      <td>16777741</td>
      <td></td>
      <td>1275</td>
      <td>gandhara_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>167</th>
      <td>33554607</td>
      <td>601</td>
      <td></td>
      <td>1334</td>
      <td>gandhara_region</td>
      <td></td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>168</th>
      <td>33554608</td>
      <td>531</td>
      <td></td>
      <td>1269</td>
      <td>syria_region</td>
      <td></td>
      <td>5</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>169</th>
      <td>67109041</td>
      <td>616</td>
      <td>[178142, 181209, 177753, 181247]</td>
      <td>1347</td>
      <td>ariana_region</td>
      <td>[1694499014]</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>170</th>
      <td>33554610</td>
      <td>624</td>
      <td>[182720, 182908, 182810, 182971]</td>
      <td>1355</td>
      <td>persis_region</td>
      <td>[1023410361]</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>171</th>
      <td>50331827</td>
      <td>614</td>
      <td>[180692, 180790, 179631, 180691]</td>
      <td>519</td>
      <td>bactriana_region</td>
      <td>[872415613]</td>
      <td>0</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>172</th>
      <td>83886260</td>
      <td>557</td>
      <td></td>
      <td>1295</td>
      <td>media_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>173</th>
      <td>33554613</td>
      <td>549</td>
      <td></td>
      <td>1287</td>
      <td>assyria_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>174</th>
      <td>182</td>
      <td>536</td>
      <td></td>
      <td>1273</td>
      <td>caledonia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>175</th>
      <td>16777399</td>
      <td>603</td>
      <td></td>
      <td>1336</td>
      <td>ariana_region</td>
      <td></td>
      <td>5</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>176</th>
      <td>184</td>
      <td>538</td>
      <td></td>
      <td>1274</td>
      <td>madhyadesa_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>177</th>
      <td>185</td>
      <td>542</td>
      <td></td>
      <td>1279</td>
      <td>pannonia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>178</th>
      <td>33554618</td>
      <td>596</td>
      <td>[175767, 180881, 179360]</td>
      <td>1262</td>
      <td>mesopotamia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>179</th>
      <td>187</td>
      <td>550</td>
      <td>[180799, 181390, 181790, 181026]</td>
      <td>1288</td>
      <td>moesia_s_region</td>
      <td>[1845493841]</td>
      <td>4</td>
      <td>3</td>
      <td>0</td>
    </tr>
    <tr>
      <th>180</th>
      <td>188</td>
      <td>552</td>
      <td></td>
      <td>1291</td>
      <td>syria_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>181</th>
      <td>16777405</td>
      <td>280</td>
      <td>[179951, 180666]</td>
      <td>26</td>
      <td>galatia_region</td>
      <td>[1962934315]</td>
      <td>7</td>
      <td>47</td>
      <td>0</td>
    </tr>
    <tr>
      <th>182</th>
      <td>16777406</td>
      <td>579</td>
      <td></td>
      <td>1314</td>
      <td>himalayan_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>183</th>
      <td>191</td>
      <td>556</td>
      <td></td>
      <td>1294</td>
      <td>albania_region</td>
      <td></td>
      <td>4</td>
      <td>5</td>
      <td>0</td>
    </tr>
    <tr>
      <th>184</th>
      <td>192</td>
      <td>558</td>
      <td>[179735, 180087, 182122]</td>
      <td>1296</td>
      <td>persis_region</td>
      <td>[721420552]</td>
      <td>3</td>
      <td>8</td>
      <td>0</td>
    </tr>
    <tr>
      <th>185</th>
      <td>33554626</td>
      <td>626</td>
      <td></td>
      <td>1357</td>
      <td>madhyadesa_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>186</th>
      <td>195</td>
      <td>564</td>
      <td></td>
      <td>1301</td>
      <td>caledonia_region</td>
      <td></td>
      <td>5</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>187</th>
      <td>196</td>
      <td>565</td>
      <td></td>
      <td>1302</td>
      <td>pracya_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>188</th>
      <td>16777413</td>
      <td>569</td>
      <td></td>
      <td>1305</td>
      <td>moesia_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>189</th>
      <td>198</td>
      <td>567</td>
      <td></td>
      <td>1303</td>
      <td>persis_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>190</th>
      <td>16777416</td>
      <td>574</td>
      <td></td>
      <td>1309</td>
      <td>assyria_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>191</th>
      <td>201</td>
      <td>571</td>
      <td></td>
      <td>1307</td>
      <td>cilicia_region</td>
      <td></td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>192</th>
      <td>16777418</td>
      <td>575</td>
      <td></td>
      <td>1310</td>
      <td>syria_region</td>
      <td></td>
      <td>1</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>193</th>
      <td>203</td>
      <td>572</td>
      <td></td>
      <td>1308</td>
      <td>punt_region</td>
      <td></td>
      <td>2</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>194</th>
      <td>204</td>
      <td>573</td>
      <td></td>
      <td>1306</td>
      <td>aparanta_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>195</th>
      <td>205</td>
      <td>577</td>
      <td></td>
      <td>1312</td>
      <td>armenia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>196</th>
      <td>206</td>
      <td>578</td>
      <td></td>
      <td>1313</td>
      <td>media_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>197</th>
      <td>16777423</td>
      <td>583</td>
      <td></td>
      <td>1316</td>
      <td>moesia_s_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>198</th>
      <td>16777424</td>
      <td>489</td>
      <td>[180331]</td>
      <td>26</td>
      <td>mesopotamia_region</td>
      <td>[1090519290]</td>
      <td>0</td>
      <td>7</td>
      <td>0</td>
    </tr>
    <tr>
      <th>199</th>
      <td>210</td>
      <td>587</td>
      <td></td>
      <td>1320</td>
      <td>lower_nubia_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>200</th>
      <td>211</td>
      <td>589</td>
      <td>[180036, 179872, 178602, 179898]</td>
      <td>1323</td>
      <td>arabia_region</td>
      <td>[2214592660]</td>
      <td>2</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <th>201</th>
      <td>212</td>
      <td>590</td>
      <td>[178008, 180719, 182292, 182658]</td>
      <td>1324</td>
      <td>arabia_felix_region</td>
      <td>[2130706612]</td>
      <td>3</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>202</th>
      <td>213</td>
      <td>591</td>
      <td></td>
      <td>1325</td>
      <td>ariana_region</td>
      <td></td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>203</th>
      <td>16777430</td>
      <td>16777810</td>
      <td></td>
      <td>1328</td>
      <td>parthia_region</td>
      <td></td>
      <td>4</td>
      <td>0</td>
      <td>0</td>
    </tr>
    <tr>
      <th>204</th>
      <td>216</td>
      <td>595</td>
      <td></td>
      <td>1329</td>
      <td>nubia_region</td>
      <td></td>
      <td>4</td>
      <td>0</td>
      <td>0</td>
    </tr>
  </tbody>
</table>